In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

import mesa
import mesa_geo as mg
import mesa_geo.visualization as mgv

from pathlib import Path

from model import GeoSIR
from utils import InfecStatus

# from model import GeoSIR
# from agents import PersonAgent, TractAgent
# from utils import TimeBlock, Decision, InfecStatus

# %load_ext autoreload
# %autoreload 2

In [2]:
data_path = Path().resolve() / 'data'
images_path = Path().resolve() / 'images'
images_path.mkdir(exist_ok=True)
flow_path = data_path / 'flow'

In [3]:
tracts_df = gpd.read_file(data_path / 'tracts.shp').to_crs(epsg=3857)
agents_tract_df = pd.read_csv(data_path / 'agents_tract.csv')
prob_stay_at_home_data = (
  pd.read_csv(
    data_path / 'agents_home.csv',
    parse_dates=['date'],
    date_format='%Y-%m-%d'
  )
  .assign(date=lambda x: x['date'].dt.date)
  .set_index(['date', 'tract'])
  .squeeze()
)
percentage_time_at_home_data = (
  pd.read_csv(
    data_path / 'agents_percentage_home.csv',
    parse_dates=['date'],
    date_format='%Y-%m-%d'
  )
  .assign(
    date=lambda x: x['date'].dt.date,
    percentage_time_home=lambda x: x['percentage_time_home'] / 100
  )
  .set_index(['date', 'tract'])
  .squeeze()
)

In [4]:
# Model
infection_params = {
  'beta': 0.5,
  'gamma': 1/2
}
initial_condition = {
  InfecStatus.S: 0.99,
  InfecStatus.I: 0.01,
  InfecStatus.R: 0.00
}
exposure_distance = 100
model = GeoSIR(
  infection_params=infection_params,
  initial_condition=initial_condition,
  exposure_distance=exposure_distance,
  tracts_df=tracts_df,
  agents_tract_df=agents_tract_df,
  prob_stay_at_home_data=prob_stay_at_home_data,
  percentage_time_at_home_data=percentage_time_at_home_data,
  flow_path=flow_path,
  seed=42
)

Initializing Time
Initializing Tracts
Initializing Population
Model ready!


In [5]:
for _ in range(30):
  model.step()

step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30


In [6]:
model.datacollector.get_model_vars_dataframe()

,date,time_block,S,I,R
0,2019-01-07,1,10320,122,0
1,2019-01-07,2,10318,124,0
2,2019-01-07,3,10310,132,0
3,2019-01-08,1,10291,151,0
4,2019-01-08,2,10280,162,0
5,2019-01-08,3,10272,59,111
6,2019-01-09,1,10248,70,124
7,2019-01-09,2,10239,71,132
8,2019-01-09,3,10231,72,139
9,2019-01-10,1,10193,87,162


In [ ]:
model._agents_by_type[PersonAgent]

In [ ]:
dir(model)

In [ ]:
Decision['STAY_HOME']

In [ ]:
model._agents_by_type[PersonAgent][0].decision

In [ ]:
model.agent_types

In [ ]:
model.datacollector.get_model_vars_dataframe()


In [ ]:
model.counts

In [ ]:
for klass, l in model._agents_by_type.items():
  print(klass, len(l))

In [ ]:
model._agents_by_type[PersonAgent]#.groupby('status').count()

In [ ]:
model.flow_dates[-1:0]

In [ ]:
model.time_block

In [ ]:
model.time_block is TimeBlock.EVENING

In [ ]:
type(model._agents_by_type[PersonAgent][0].geometry)

In [ ]:
model._agents_by_type[PersonAgent][0].pos

In [ ]:
model._agents_by_type[TractAgent][0].prob_stay_at_home

In [ ]:
model.today

In [ ]:
# (
#   model.tracts_df.merge(
#     model.agents_tract_df,
#     how='left',
#     on='tract',
#     validate='1:1'
#   )
#   .rename(columns={'tract': 'code', 'n_agents': 'population'})
# )

In [ ]:
set(InfecStatus)

In [ ]:
InfecStatus.S in InfecStatus

In [ ]:
InfecStatus

In [ ]:
InfecStatus.S in set(InfecStatus)

In [ ]:
type(InfecStatus)

In [ ]:
members = list(TimeBlock)

In [ ]:
members.index(TimeBlock.MORNING)

In [ ]:
counts = {
      "S": 0,
      "I": 0,
      "R": 0,
    }

In [ ]:
InfecStatus.S in counts.keys()

In [ ]:
counts = {status: 0 for status in InfecStatus}
counts

In [ ]:
counts.keys()

In [ ]:
list(InfecStatus)

In [ ]:
InfecStatus.__members__.values()

In [ ]:
counts[InfecStatus.S]

In [ ]:
print(InfecStatus.S in counts)

In [ ]:
from enum import Flag, auto

class InfecStatus(Flag):
    S = auto()
    I = auto()
    R = auto()

counts = {status: 0 for status in InfecStatus}
print(counts[InfecStatus.S])  # Should work

In [ ]:
model.prob_stay_at_home_data

In [ ]:
model.today

In [ ]:
model.prob_stay_at_home_data.index.dtype

In [ ]:
my_date = percentage_time_at_home_data.index.get_level_values(0)[0]

In [ ]:
model.prob_stay_at_home_data.loc[model.current_flow_date, 53033032326]

In [ ]:
model.current_flow_date

In [ ]:
model._agents_by_type[PersonAgent][0].tract_id

In [ ]:
model.agents[1].prob_stay_at_home

In [ ]:
np.random.rand()

In [ ]:
tract_id = 1
tract = model.agents[tract_id]
tract.prob_flow

In [ ]:
model.random.choices(list(tract.prob_flow.keys()), weights=list(tract.prob_flow.values()))[0]

In [ ]:
InfecStatus._member_names_

In [ ]:
model.agents[1].geometry

In [ ]:
gpd.GeoSeries(model.agents[1].geometry).sample_points(size=(1, 2)).iat[0]

In [ ]:
model._agents